# Restaurant Performance & Menu Optimization Analysis

# Project Overview

This project analyzes a quarter’s worth of transactional order data from a fictitious restaurant serving international cuisine to evaluate menu performance, revenue drivers, and operational efficiency. The goal is to move beyond descriptive insights and deliver clear, data-backed recommendations that improve profitability and operational decision-making.

Using transaction-level order data, this analysis identifies underperforming menu items, high-value orders, peak and off-peak demand periods, and cuisine-level opportunities for menu optimization and staffing alignment.

# Business Problem

Restaurants often face margin pressure due to menu bloat, inefficient staffing, and misalignment between customer demand and operational resources. Without data-driven insights, decisions around menu changes and staffing levels are typically based on intuition rather than evidence.

This project addresses the following core questions:

Which menu items and categories truly drive value?

Where are resources being allocated inefficiently?

How can menu and staffing decisions be optimized to improve profitability?

# Project Objectives

1. Identify high- and low-performing menu items based on order volume and revenue contribution.

2. Analyze category- and cuisine-level performance to uncover demand patterns.

3. Examine high-spend orders to understand purchasing behavior and revenue concentration.

4. Evaluate temporal ordering trends to identify peak and off-peak periods.

5. Assess operational efficiency by aligning order volume with staffing demand.

6. Provide actionable recommendations for menu optimization and staffing adjustments.

In [1]:
#Import necessary modules
import pandas as pd
import os
import sqlite3

In [2]:
# Connect to the SQLite database
db_path = os.path.join(os.getcwd(), 'restaurant_performance.db')
conn = sqlite3.connect(db_path)
cursor = conn.cursor()


In [5]:
# Create tables and load data from CSV files
# Create menu_items table
cursor.execute("""
    CREATE TABLE IF NOT EXISTS menu_items (
        menu_item_id INTEGER PRIMARY KEY,
        item_name TEXT,
        category TEXT,
        price REAL
    )
""")

# Create order_details table
cursor.execute("""
    CREATE TABLE IF NOT EXISTS order_details (
        order_details_id INTEGER PRIMARY KEY,
        order_id INTEGER,
        order_date DATE,
        order_time TIME,
        item_id INTEGER
    )
""")

# Load menu_items from CSV
menu_df = pd.read_csv('Restaurant Orders/menu_items.csv')
menu_df.to_sql('menu_items', conn, if_exists='replace', index=False)

# Load order_details from CSV
order_details_df = pd.read_csv('Restaurant Orders/order_details.csv')
order_details_df.to_sql('order_details', conn, if_exists='replace', index=False)

conn.commit()
print("✓ Tables created and data loaded successfully!")

✓ Tables created and data loaded successfully!


In [6]:
# Verify data loaded correctly
cursor.execute("SELECT COUNT(*) FROM order_details")
order_count = cursor.fetchone()[0]

cursor.execute("SELECT COUNT(*) FROM menu_items")
menu_count = cursor.fetchone()[0]

cursor.execute("SELECT MIN(order_date), MAX(order_date) FROM order_details")
date_range = cursor.fetchone()

print(f"Order Details Records: {order_count:,}")
print(f"Menu Items Records: {menu_count}")
print(f"Date Range: {date_range[0]} to {date_range[1]}")

Order Details Records: 12,234
Menu Items Records: 32
Date Range: 1/1/23 to 3/9/23


In [7]:
# Menu Items performance overview
menu_df = pd.read_sql_query("SELECT * FROM menu_items", conn)

print("Menu Items Summary:")
print(f"Total unique menu items: {len(menu_df)}")
print(f"\nMenu Items by Category:")
print(menu_df['category'].value_counts())
print(f"\nPrice Range: ${menu_df['price'].min()} - ${menu_df['price'].max()}")
print(f"\nMenu items with missing data:")
print(menu_df[menu_df.isnull().any(axis=1)])

Menu Items Summary:
Total unique menu items: 32

Menu Items by Category:
category
Mexican     9
Italian     9
Asian       8
American    6
Name: count, dtype: int64

Price Range: $5.0 - $19.95

Menu items with missing data:
Empty DataFrame
Columns: [menu_item_id, item_name, category, price]
Index: []


In [8]:
# Check for missing values and data quality issues
print("=" * 60)
print("MISSING VALUES ANALYSIS")
print("=" * 60)

# Missing values in order_details
cursor.execute("""
    SELECT 
        COUNT(*) as total_records,
        COUNT(CASE WHEN item_id IS NULL THEN 1 END) as null_item_ids,
        COUNT(CASE WHEN order_date IS NULL THEN 1 END) as null_dates,
        COUNT(CASE WHEN order_time IS NULL THEN 1 END) as null_times
    FROM order_details
""")
result = cursor.fetchone()
print(f"\norder_details table:")
print(f"  Total records: {result[0]:,}")
print(f"  NULL item_ids: {result[1]} ({result[1]/result[0]*100:.2f}%)")
print(f"  NULL order_dates: {result[2]}")
print(f"  NULL order_times: {result[3]}")


MISSING VALUES ANALYSIS

order_details table:
  Total records: 12,234
  NULL item_ids: 137 (1.12%)
  NULL order_dates: 0
  NULL order_times: 0


In [9]:

# Missing values in menu_items
cursor.execute("""
    SELECT 
        COUNT(*) as total_items,
        COUNT(CASE WHEN item_name IS NULL THEN 1 END) as null_names,
        COUNT(CASE WHEN category IS NULL THEN 1 END) as null_categories,
        COUNT(CASE WHEN price IS NULL THEN 1 END) as null_prices
    FROM menu_items
""")
result = cursor.fetchone()
print(f"\nmenu_items table:")
print(f"  Total items: {result[0]}")
print(f"  NULL item_names: {result[1]}")
print(f"  NULL categories: {result[2]}")
print(f"  NULL prices: {result[3]}")


menu_items table:
  Total items: 32
  NULL item_names: 0
  NULL categories: 0
  NULL prices: 0


In [10]:
# Check for invalid data - prices, duplicates, unmatched items
print("\n" + "=" * 60)
print("DATA QUALITY CHECKS")
print("=" * 60)

# Price validation
cursor.execute("SELECT COUNT(*), MIN(price), MAX(price) FROM menu_items")
count, min_price, max_price = cursor.fetchone()
print(f"\nPrice range (menu_items): ${min_price:.2f} - ${max_price:.2f}")



DATA QUALITY CHECKS

Price range (menu_items): $5.00 - $19.95


In [11]:

# Check for items in orders that don't exist in menu_items
cursor.execute("""
    SELECT COUNT(*) as unmatched_items
    FROM order_details od
    WHERE od.item_id NOT IN (SELECT menu_item_id FROM menu_items)
    AND od.item_id IS NOT NULL
""")
unmatched = cursor.fetchone()[0]
print(f"Orders referencing non-existent items: {unmatched}")


Orders referencing non-existent items: 0


In [12]:

# Duplicate check
cursor.execute("SELECT COUNT(*), COUNT(DISTINCT order_details_id) FROM order_details")
total, distinct = cursor.fetchone()
print(f"Duplicate order_details_ids: {total - distinct}")

cursor.execute("SELECT COUNT(*), COUNT(DISTINCT menu_item_id) FROM menu_items")
total, distinct = cursor.fetchone()
print(f"Duplicate menu_item_ids: {total - distinct}")


Duplicate order_details_ids: 0
Duplicate menu_item_ids: 0


In [13]:

# Sample of records with NULL item_ids
print("\nSample records with NULL item_ids:")
cursor.execute("SELECT order_details_id, order_id, order_date FROM order_details WHERE item_id IS NULL LIMIT 5")
for row in cursor.fetchall():
    print(f"  - order_details_id: {row[0]}, order_id: {row[1]}, date: {row[2]}")


Sample records with NULL item_ids:
  - order_details_id: 122, order_id: 50, date: 1/1/23
  - order_details_id: 298, order_id: 125, date: 1/2/23
  - order_details_id: 358, order_id: 147, date: 1/3/23
  - order_details_id: 387, order_id: 161, date: 1/3/23
  - order_details_id: 470, order_id: 200, date: 1/3/23


In [18]:
# Summary statistics before cleaning
print("\n" + "=" * 60)
print("SUMMARY STATISTICS")
print("=" * 60)

# Unique orders
cursor.execute("SELECT COUNT(DISTINCT order_id) FROM order_details")
unique_orders = cursor.fetchone()[0]
print(f"\nUnique orders: {unique_orders:,}")


# Items per order
cursor.execute("""
    SELECT 
        AVG(items_per_order) as avg_items,
        MIN(items_per_order) as min_items,
        MAX(items_per_order) as max_items
    FROM (
        SELECT COUNT(*) as items_per_order
        FROM order_details
        WHERE item_id IS NOT NULL
        GROUP BY order_id
    )
""")
avg_items, min_items, max_items = cursor.fetchone()
print(f"Items per order: avg={avg_items:.2f}, min={min_items}, max={max_items}")


# Menu categories
cursor.execute("SELECT COUNT(DISTINCT category) FROM menu_items")
num_categories = cursor.fetchone()[0]
print(f"Menu categories: {num_categories}")

cursor.execute("""
    SELECT category, COUNT(*) as item_count
    FROM menu_items
    GROUP BY category
    ORDER BY item_count DESC
""")
print("\nItems by category:")
for row in cursor.fetchall():
    print(f"  - {row[0]}: {row[1]} items")


SUMMARY STATISTICS

Unique orders: 5,370
Items per order: avg=2.26, min=1, max=14
Menu categories: 4

Items by category:
  - Mexican: 9 items
  - Italian: 9 items
  - Asian: 8 items
  - American: 6 items
